In [3]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [4]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = 'localhost'
DB2_PORT = '51000'
DB2_USERNAME = 'mpower_user'
DB2_PASSWORD = 'secret'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=51000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=51000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

<h3>Look Up Tables</h3>

In [7]:
# Region
cols_map = [
    {'src_col': 'JUNCTION', 'des_col': 'junction', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNT_NO', 'des_col': 'account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CUSTMER_NAME', 'des_col': 'cutomer_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': '', 'des_col': 'deleted', 'src_dtype': 'String', 'des_dtype': 'Boolean', 'des_value': 'false'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'DABS.ANOMALIES_LIST', 'public', '"ANOMALIES_LIST"', cols_map)
print("Total Migrated Records (ANOMALIES_LIST): " + str(migrated_row_count))



# sql = "SELECT * FROM DABS.ANOMALIES_LIST;"
# df = pd.read_sql(sql, db2_conni)
# df.head()

Total Migrated Records (ANOMALIES_LIST): Total Src Records: 61834 Total migrated Records to dest: 123668


In [44]:
#from ibm_db import connect
# Careful with the punctuation here - we have 3 arguments.
# The first is a big string with semicolons in it.
# (Strings separated by only whitespace, newlines included,
#  are automatically joined together, in case you didn't know.)
# The last two are emptry strings.
#connection = connect('DATABASE=<database name>;'
#                     'HOSTNAME=<database ip>;'  # 127.0.0.1 or localhost works if it's local
#                     'PORT=<database port>;'
#                     'PROTOCOL=TCPIP;'
#                     'UID=<database username>;'
#                     'PWD=<username password>;', '', '')

ibm_db never actually give us results. Instead, we need to call one of the fetch methods on the command, repeatedly, to get the results this helper function to deal with that.

In [5]:
def fetch_db2_results(command):
    from ibm_db import fetch_assoc

    ret = []
    result = fetch_assoc(command)
    while result:
        # This builds a list in memory. Theoretically, if there's a lot of rows,
        # we could run out of memory. In practice, I've never had that happen.
        # If it's ever a problem, you could use
        #     yield result
        # Then this function would become a generator. You lose the ability to access
        # results by index or slice them or whatever, but you retain
        # the ability to iterate on them.
        ret.append(result)
        result = fetch_assoc(command)
    return ret  # Ditch this line if you choose to use a generator.

we can easily do something like get the information on all the tables in your database with the following:

In [6]:
from ibm_db import tables

t = fetch_db2_results(tables(connection))

NameError: name 'connection' is not defined

Execute a sql command on DB2

In [7]:
from ibm_db import exec_immediate

sql = 'LIST * FROM ' + t[170]['TABLE_NAME']  # Using our list of tables t from before...
rows = fetch_db2_results(exec_immediate(connection, sql))

NameError: name 't' is not defined

### Postgres Section

In [ ]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'asims_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

# ASIMS ENDPOINT
ASIMS_API_ENDPOINT = 'http://localhost:8090/odkx/odktables/db_sync/'

# List all table names of ODKX from Sync Endpoint
odkx_db_table_names = ['NSIA_HOUSEHOLD_MALE',
                    'NSIA_HOUSEHOLD_FEMALE', 
                    'NSIA_ROSTER_MALE', 
                    'NSIA_ROSTER_FEMALE', 
                    'NSIA_LABOUR_MALE', 
                    'NSIA_CHILD_LABOUR', 
                    'NSIA_HEALTH', 
                    'SHURA', 
                    'MARKET_PRICE', 
                    'WATER_TESTING', 
                    'NSIA_CAPI', 
                    'NSIA_CAPI_LIST']
odkx_db_table_prefix = '__ODKTABLES__TABLE_'
odkx_db_schema = 'odk_sync.'